In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=["result", "sms"])
test_df  = pd.read_csv(test_file_path,  sep='\t', header=None, names=["result", "sms"])

In [ ]:
test_df.head()

In [ ]:
def tokenize(text):
    text = text.replace("'", '')
    text = text.replace(",", '')
    text = text.replace("!", '')
    text = text.replace("*", '')
    text = text.replace('.', '').lower()
    return text.split()

In [ ]:
def encode_label(x):
    if x == 'ham':
        return 0
    else:
        return 1


train_df['label'] = train_df['result'].apply(encode_label)
test_df['label'] = test_df['result'].apply(encode_label)

In [ ]:
vocab = {"0":0}

num_words = 1
for index, row in train_df.iterrows():
    for word in tokenize(row['sms']):
        if not word in vocab:
            vocab[word] = num_words
            num_words += 1

for index, row in test_df.iterrows():
    for word in tokenize(row['sms']):
        if not word in vocab:
            vocab[word] = num_words
            num_words += 1

In [ ]:
vocab['dear']

In [ ]:
def encode_text(text):
    tokens = tokenize(text)
    ecoded_text = [vocab.get(word, 0) for word in tokens]
    return ecoded_text

In [ ]:
train_df['encode_text'] = train_df['sms'].apply(encode_text)
test_df['encode_text'] = test_df['sms'].apply(encode_text)

In [ ]:
train_df.head()

In [ ]:
import numpy as np

# Get list of sequence lengths
lengths = train_df['encode_text'].apply(len)

# Calculate percentiles
p70 = np.percentile(lengths, 70)
p80 = np.percentile(lengths, 80)
p90 = np.percentile(lengths, 90)
p99 = np.percentile(lengths, 99)

print(f"70th percentile length: {p70}")
print(f"80th percentile length: {p80}")
print(f"90th percentile length: {p90}")
print(f"99th percentile length: {p99}")

**If I wish to cover 99% percent of the data then maxlen should be 54**

In [ ]:
from tensorflow.keras.utils import pad_sequences

maxlen = 54

train_df['padded_text'] = pad_sequences(
    train_df['encode_text'].tolist(),  # Convert Series to list of lists
    maxlen=maxlen,
    padding='pre',       # or 'post' if you prefer
    truncating='pre',    # or 'post'
    value=0              # Use 0 for padding token ID
).tolist()  # Optional: convert back to list so it fits nicely in a DataFrame

test_df['padded_text'] = pad_sequences(
    test_df['encode_text'].tolist(),  # Convert Series to list of lists
    maxlen=maxlen,
    padding='pre',       # or 'post' if you prefer
    truncating='pre',    # or 'post'
    value=0              # Use 0 for padding token ID
).tolist()  # Optional: convert back to list so it fits nicely in a DataFrame



In [ ]:
train_df.head()

In [ ]:
import numpy as np

X = np.array(train_df['padded_text'].tolist())
y = np.array(train_df['label'].tolist())

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = len(vocab) + 1  # +1 if you used 0 for padding
embedding_dim = 50
maxlen = X.shape[1]  # padded sequence length

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),
    LSTM(64),
    Dense(1, activation='sigmoid')  # For binary classification
])


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=5, validation_split=0.2)


In [ ]:
X_test = np.array(test_df['padded_text'].tolist())
y_test = np.array(test_df['label'].tolist())

model.evaluate(X_test, y_test)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    possible = ["ham", 'spam']
    encoded_text = encode_text(pred_text)
    padded_text = pad_sequences([encoded_text], maxlen=maxlen, padding='pre', truncating='pre', value=0)
    prediction = model.predict(padded_text)
    result = possible[(prediction[0]> 0.5).astype("int32")[0]]
    return result

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
